# సెషన్ 6 – బహుళ దశల పైప్‌లైన్ (ప్లాన్ → అమలు → మెరుగుపరచు)

ప్రతి దశకు మోడల్స్‌ను ఎంచుకోవడానికి రౌటింగ్ ఫంక్షన్లను ఉపయోగించి, ఆపై తుది సమాధానాన్ని మెరుగుపరుస్తుంది.


### వివరణ: డిపెండెన్సీ ఇన్‌స్టాలేషన్
ప్రతి దశ చాట్ కాల్స్ కోసం అవసరమైన `foundry-local-sdk` మరియు `openai` ను ఇన్‌స్టాల్ చేస్తుంది. మళ్లీ రన్ చేయడం సురక్షితం.


# సన్నివేశం
బహుళ-దశ మోడల్ పైప్‌లైన్ ఇది: (1) ఒక పనిని విడివిడిగా దశలుగా ప్రణాళిక చేస్తుంది, (2) ప్రతి దశను ఉద్దేశ్య ఆధారిత మోడల్ ఎంపికతో అమలు చేస్తుంది, (3) తుది సంశ్లేషిత సమాధానాన్ని మెరుగుపరుస్తుంది. నాణ్యతను కాపాడుకుంటూ సామర్థ్యాన్ని పెంచేందుకు విభిన్న SLM సామర్థ్యాలను చైన్ చేయడం చూపిస్తుంది.


In [13]:
!pip install -q foundry-local-sdk openai

### వివరణ: కోర్ ఇంపోర్ట్స్  
ఇంటెంట్ డిటెక్షన్ కోసం regex, ప్రతి అలియాస్ అటాచ్మెంట్ కోసం Foundry Local మేనేజర్, మరియు చాట్ కంప్లీషన్స్ కోసం OpenAI క్లయింట్‌ను ఇంపోర్ట్ చేస్తుంది.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### వివరణ: సామర్థ్య క్యాటలాగ్ & నియమాలు
స్టెప్ టెక్స్ట్‌ను ఉద్దేశ్య వర్గాలకు (కోడ్, సారాంశం, వర్గీకరణ, సాధారణ) మ్యాప్ చేయడానికి ఉపయోగించే సామర్థ్య-అవగాహన క్యాటలాగ్ మరియు రెగ్యులర్ ఎక్స్‌ప్రెషన్ నియమాలను నిర్వచిస్తుంది. ఒక ఉద్దేశ్యాన్ని అనేక మోడల్స్ మద్దతు ఇచ్చినప్పుడు తక్కువ ప్రాధాన్యత విలువలు సమతుల్యతను గెలుస్తాయి.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### వివరణ: ఉద్దేశ్యం, మోడల్ ఎంపిక & చాట్ సహాయకుడు
ఇది అందిస్తుంది:
- regex-ఆధారిత వర్గీకరణ కోసం `detect_intent`.
- సామర్థ్యం + ప్రాధాన్యత ఆధారంగా ఉత్తమ అలియాస్‌ను ఎంచుకోవడానికి `pick_model`.
- ప్రతి అలియాస్‌కు క్లయింట్‌ను సృష్టించి ఒకే-తిరుగుబాటు ప్రతిస్పందనను అందించే సౌకర్యవంతమైన `chat` ర్యాపర్.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### వివరణ: బహుళ-దశ పైప్‌లైన్ ఫంక్షన్
పైప్‌లైన్‌ను అమలు చేస్తుంది: ప్రణాళిక → దశలను విశ్లేషించు → ఉద్దేశ్య ఆధారిత రూటింగ్‌తో ప్రతి దశను అమలు చేయు → కలిపిన అవుట్‌పుట్‌లను మెరుగుపరచు. పరిశీలన లేదా మూల్యాంకన కోసం నిర్మిత డిక్ట్‌ను తిరిగి ఇస్తుంది.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### వివరణ: ఉదాహరణ టాస్క్ నడపండి
కోడ్ + సారాంశం వంటి మిశ్రమ ఉద్దేశాలను చూపించే రిఫాక్టర్-ఆధారిత టాస్క్‌పై పూర్తి పైప్‌లైన్‌ను అమలు చేస్తుంది. ఫలిత డిక్ట్‌లో ముడి ప్రణాళిక, ప్రతి దశ అవుట్‌పుట్‌లు ఎంపిక చేసిన మోడల్ అలియాసులతో, మరియు తుది సంశ్లేషించిన సమాధానం చూపబడుతుంది.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### వివరణ: ఫలిత ఆబ్జెక్ట్ ప్రదర్శన
త్వరిత పరిశీలన లేదా దిగువన మూల్యాంకన కోసం నిర్మితమైన పైప్‌లైన్ అవుట్‌పుట్‌ను చూపిస్తుంది (ఉదాహరణకు, దశ నాణ్యత లేదా శుద్ధి ప్రభావాన్ని కొలవడం).


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారుల కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
